## Universidad Nacional de Colombia

## Diplomado Ciencia de datos


## Clusterización

En este caso práctico aplicaremos el algoritmos de clusterización para hacer agrupamiento de datos, aprenderemos cómo funcionan y cómo utilizarlos en un caso práctico

El caso estará estructurado así
1. Hacer un análisis exploratorio para revisar la estructura de los datos
2. Usar lo que observamos para guiar nuestro proceso de agrupamiento
3. Ajustar un algortimo de k-means a los datos disponibles
4. Ajustar un agrupamiento jerárquico
5. Hacer segmentaciones y conclusiones a partir del análisis

**Contexto:** Las competencias deportivas cada día recogen una gran cantidad de datos relacionados con el desempeño de sus equipos y jugadores para encontrar patrones en estos datos y tomar decisiones informadas basadas en ellos. De esta manera la competencia aumenta tanto dentro como fuera de la cancha

**Problema de negocio:** Se tienen los datos de desempeño de los equipos de baloncesto del torneo NCAA March Madness que contiene las estadísticas de juego de 353 equipos de la liga. El objetivo es inspeccionar esta data utilizando técnicas de visualización y agrupación para encontrar patrones en el desempeño de los equipos y generar recomendaciones de umbrales en las estadísticas para que un equipo esté en el grupo de desempeño superior.

In [ ]:
# librerías
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
import sklearn
from scipy.stats import norm
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, RobustScaler, StandardScaler
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering 
from sklearn.datasets.samples_generator import make_blobs
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
from matplotlib import pyplot as plt

In [ ]:
# display setting Para visualizar el máximo de columnas
pd.set_option('display.max_columns', None)

In [ ]:
datos = pd.read_csv('basketball_19.csv')

In [ ]:
datos.columns

Estas son las variables que contiene el conjunto de datos 

- TEAM: Equipo
- CONF: La conferencia en la que el equipo participa(A10 = Atlantic 10, ACC = Atlantic Coast Conference, AE = America East, Amer = American, ASun = ASUN, B10 = Big Ten, B12 = Big 12, BE = Big East, BSky = Big Sky, BSth = Big South, BW = Big West, CAA = Colonial Athletic Association, CUSA = Conference USA, Horz = Horizon League, Ivy = Ivy League, MAAC = Metro Atlantic Athletic Conference, MAC = Mid-American Conference, MEAC = Mid-Eastern Athletic Conference, MVC = Missouri Valley Conference, MWC = Mountain West, NEC = Northeast Conference, OVC = Ohio Valley Conference, P12 = Pac-12, Pat = Patriot League, SB = Sun Belt, SC = Southern Conference, SEC = South Eastern Conference, Slnd = Southland Conference, Sum = Summit League, SWAC = Southwestern Athletic Conference, WAC = Western Athletic Conference, WCC = West Coast Conference)
- G: Número de partidos jugados
- W: Número de partidos ganados
- ADJOE: Estimación de eficiencia ofensiva, puntos anotados por cada 100 posesiones
- ADJDE: Estimación de eficiencia defensiva, puntos permitidos por cada 100 posesiones del equipo contrario
- BARTHAG: Probabilidad de vencer a un equipo
- EFG_O: Effective Field Goal Percentage Shot
- EFG_D: Effective Field Goal Percentage Allowed
- TOR: Porcentaje de rotación permitida (equipo pierde la posesión del balón contra el equipo contrario antes de que un jugador dispare a la canasta de su equipo)
- TORD: Porcentaje de rotación hecha al equipo contrario (se roba la pelota al contrincante)
- ORB: Porcentaje de rebote ofensivo
- B: Porcentaje de rebote defensivo
- FTR : Tasa de tiros libres hechos(que hace el equipo)
- FTRD: Tasa de tiros libres permitidos (que hace el contrincante)
- 2P_O: Porcentaje de tiros de 2 puntos hechos
- 2P_D: Porcentaje de tiros de 2 puntos permitidos
- 3P_O: Porcentaje de tiros de 3 puntos hechos
- 3P_D: Porcentaje de tiros de 3 puntos permitidos
- ADJ_T: Posesión del balón por 40 min
- WAB: Triunfos por encima de la 'burbuja' (la burbuja es el límite definido para pasar al campeonato NCAA March Madness Tournament
- POSTSEASON: Ronda en la que el equipo de fue eliminado (R68 = First Four, R64 = Round of 64, R32 = Round of 32, S16 = Sweet Sixteen, E8 = Elite Eight, F4 = Final Four, 2ND = Runner-up, Champion = Winner of the NCAA March Madness Tournament for that given year)
- SEED: Semilla definida por el torneo


In [ ]:
datos.head()

## Exploración de los datos 

Para empezar el análisis hay que hacer una exploración inicial de los datos, entender un poco las variables y la información que tenemos. Para empezar nuestros datos consisten en las estadísticas de 353 equipos contenidas en 24 variables 

In [ ]:
datos.shape

In [ ]:
datos.describe()

Visualicemos el comportamiento de algunas variables

In [ ]:
columnas = ['G', 'W', 'ADJOE', 'ADJDE', 'BARTHAG', 'EFG_O', 'EFG_D',
       'TOR', 'TORD', 'ORB', 'DRB', 'FTR', 'FTRD', '2P_O', '2P_D', '3P_O',
       '3P_D', 'ADJ_T', 'WAB']

In [ ]:
fig=plt.figure(figsize=(20,30))
for i, feature in enumerate(columnas):
    ax=fig.add_subplot(10,2,i+1)
    sns.distplot(datos[feature], bins=14, kde=False)
    ax.set_title(feature+" Distribution")

fig.tight_layout()  
plt.show()

Podríamos empezar a identificar cuál es el comportamiento de los equipos durante la temporada 

In [ ]:
plt.figure(figsize=(8,6))

plt.scatter(datos['BARTHAG'], datos['W'],alpha=0.5, edgecolor='k')
plt.title('Relación partidos ganados con probabilidad de vencer un equipo ', fontsize=16)
plt.xlabel('Probabilidad de ganar', fontsize=14)
plt.xticks(fontsize=12)
plt.ylabel('Partidos ganados', fontsize=14)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
# Cómo se relacionan algunas variables con la probabilidad de ganar
plt.figure(figsize=(12,10))

plt.subplot(321)
plt.scatter(y=datos['BARTHAG'], x=datos['ADJOE'],alpha=0.5, edgecolor='k')
plt.yticks(fontsize=12)
plt.ylabel('Prob Ganar', fontsize=12)
plt.title('Eficiencia Ofensiva', fontsize=16)

plt.subplot(322)
plt.scatter(y=datos['BARTHAG'], x=datos['EFG_O'],alpha=0.5, edgecolor='k')
plt.yticks(fontsize=12)
plt.ylabel('Prob Ganar', fontsize=12)
plt.title('% Tiros efectivos', fontsize=16)


plt.subplot(323)
plt.scatter(y=datos['BARTHAG'], x=datos['ORB'],alpha=0.5, edgecolor='k')
plt.yticks(fontsize=12)
plt.ylabel('Prob Ganar', fontsize=12)
plt.title('% rebote ofensivo', fontsize=16)


# Partidos ganados
plt.subplot(324)
plt.scatter(y=datos['BARTHAG'], x=datos['TOR'],alpha=0.5, edgecolor='k')
plt.ylabel('Prob Ganar', fontsize=12)
plt.yticks(fontsize=12)
plt.title('% rotación', fontsize=16)


plt.subplot(325)
plt.scatter(y=datos['BARTHAG'], x=datos['2P_O'],alpha=0.5, edgecolor='k')
plt.ylabel('Prob ganar', fontsize=12)
plt.yticks(fontsize=12)
plt.title('% tiros de 2 puntos hechos', fontsize=16)


plt.subplot(326)
plt.scatter(y=datos['BARTHAG'], x=datos['ADJ_T'],alpha=0.5, edgecolor='k')
plt.ylabel('Prob ganar', fontsize=12)
plt.yticks(fontsize=12)
plt.title('Posesión del balón', fontsize=16)


plt.show()

Inspeccionemos las correlaciones existentes entre las variables numéricas

In [ ]:
km_data = datos.drop(['TEAM','CONF','POSTSEASON','SEED'],axis=1)

In [ ]:
corr = km_data.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

## Agrupamiento 

A partir del análisis exploratorio, escogemos algunas variables que parecen tener una mayor variabilidad y nos permitirán identificar tipos de equipos

In [ ]:
km = km_data[['W','ADJOE','BARTHAG','EFG_O','2P_O','WAB']]
km.head()

Al igual que en ACP es importante estandarizar las variables que vamos a utilizar. La función **StandardScaler** nos permite hacerlo en una sola linea

In [ ]:
scaler = StandardScaler()
km_scale = scaler.fit_transform(km)
km_scale[0:5]

## Agrupamiento k-means

Utilizaremos la funcion KMeans de la librería sklearn https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

Los principales argumentos que debemos dar a la función son:
- n_clusters, al ser un método particional debemos definir el número de clusters
- init, indica el método con el cuál vamos a escoger los puntos de inicio de los centroides 
- random_state, definir una semilla para poder reproducir 


Primero definimos los parámetros del algoritmo

In [ ]:
kmeans = KMeans(
        init="random",
        n_clusters=3,
        random_state=42
    )

Ajustamos el algoritmo a los datos

In [ ]:
kmeans.fit(km_scale)

Los atributos generados por el algoritmo son:

- cluster_centers_, los centroides de los clusters encontrados 
- labels_, la asignación de cluster para cada punto
- inertia_, suma de distancias cuadradas de cada punto a su centroide-Total Suma de cuadrados dentro  (comparar métodos)

In [ ]:
print('La suma de distancias cuadradas de cada punto a su centroide en esta solución es de')
print(kmeans.inertia_)
print('Pero este número por si solo no es muy explicativo')

**Centroides**

Tener en cuenta que las variables están estandarizadas, no necesariamente estos valores tengan un significado explícito pero sus magnitudes y sentidos si nos pueden decir mucho sobre las variables originales

In [ ]:
centroides = pd.DataFrame(kmeans.cluster_centers_)
centroides.columns = ['W','ADJOE','BARTHAG','EFG_O','2P_O','WAB']
centroides

Podríamos llegar a decir que los equipos del cluster 2 son quienes más han ganado partidos, mientras que los del cluster 0 son los del menor número de partidos ganados.

Revisemos cómo podemos relacionar el cluster con las variables originales

Guardamos la asignación del cluster en una nueva columna del data set

In [ ]:
datos['cluster'] = kmeans.labels_

Y podemos visualizar cómo se comportan los clusters con respecto a las variables originales

In [ ]:
plt.figure(figsize=(12,10))

sns.boxplot(y=datos['W'],x=datos['cluster'])
plt.yticks(fontsize=12)
plt.title('Partidos ganados según cluster', fontsize=16)

In [ ]:
datos.groupby('cluster')['W','ADJOE','BARTHAG','EFG_O','2P_O','WAB'].mean()

In [ ]:
# Relacion con probabilidad de ganar por cluster
plt.figure(figsize=(12,10))

plt.subplot(321)
plt.scatter(y=datos['BARTHAG'], x=datos['W'],c=datos['cluster'],alpha=0.5, edgecolor='k')
plt.yticks(fontsize=12)
plt.ylabel('Prob Ganar', fontsize=12)
plt.title('Partidos ganados', fontsize=16)

plt.subplot(322)
plt.scatter(y=datos['BARTHAG'], x=datos['ADJOE'],c=datos['cluster'],alpha=0.5, edgecolor='k')
plt.yticks(fontsize=12)
plt.ylabel('Prob Ganar', fontsize=12)
plt.title('Eficiencia defensiva', fontsize=16)


plt.subplot(323)
plt.scatter(y=datos['BARTHAG'], x=datos['EFG_O'],c=datos['cluster'],alpha=0.5, edgecolor='k')
plt.yticks(fontsize=12)
plt.ylabel('Prob Ganar', fontsize=12)
plt.title('Lanzamientos efectivos', fontsize=16)


# Partidos ganados
plt.subplot(324)
plt.scatter(y=datos['BARTHAG'], x=datos['2P_O'],c=datos['cluster'],alpha=0.5, edgecolor='k')
plt.ylabel('Prob Ganar', fontsize=12)
plt.yticks(fontsize=12)
plt.title('Porcentaje de tiros de 2 puntos hechos', fontsize=16)


plt.subplot(325)
plt.scatter(y=datos['BARTHAG'], x=datos['WAB'],c=datos['cluster'],alpha=0.5, edgecolor='k')
plt.ylabel('Prob ganar', fontsize=12)
plt.yticks(fontsize=12)
plt.title('Triunfos por encima de la burbuja', fontsize=16)




plt.show()

## Agrupamiento jerárquico

En el agrupamiento jerárquico primero podemos inspeccionar cuántos clusters debería considerar el algoritmo usando el dendograma. Para hacer esto utilizaremos las funciones **linkage** (https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html) y **dendogram** (https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html) de la librería **scipy**

Para el linkage utilizaremos el método ward que minimiza la varianza dentro de los clusters

In [ ]:
plt.figure(figsize=(10,8))
dendrogram = sch.dendrogram(sch.linkage(km_scale, method  = "ward"))
plt.title('Dendrogram')
plt.xlabel('Equipos')
plt.ylabel('Distancias euclideanas')
plt.show()

El gráfico nos muestra las ramificaciones de posibles clusters y nos puede indicar un buen número de clusters. Para continuar con la lógica del k-means anterior utilizaremos el agrupamiento de 3 clusters. Para hacer el ajuste del algoritmo utilizaremos la función **AgglomerativeClustering** de sklearn (https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html)

In [ ]:
jer = AgglomerativeClustering(n_clusters = 3, affinity = 'euclidean', linkage ='ward')

Guardemos el cluster asignado a cada observación

In [ ]:
datos['cluster_j']= jer.fit_predict(km_scale)

Comparemos los resultados del k-means con el jerárquico. 


**Nota:** Los algoritmos no necesariamente van a dar las mismas etiquetas a cada observación, esto incluso si se corre el mismo k-means varias veces los resultados pueden ser diferentes. Lo que queremos es analizar que tan similar es la asignación de clusters

In [ ]:
pd.crosstab(datos['cluster'],datos['cluster_j'])

Podríamos inferir que el cluster 0 en el k-means tiene una composición similar para el cluster 2 en el jerárquico, y viceversa. Revisemos cómo se comportan los clusters con respecto a las variables observadas 

In [ ]:
plt.figure(figsize=(12,10))
sns.boxplot(y=datos['W'],x=datos['cluster_j'])
plt.yticks(fontsize=12)
plt.title('Partidos ganados según cluster jerárquico', fontsize=16)

In [ ]:
datos.groupby('cluster_j')['W','ADJOE','BARTHAG','EFG_O','2P_O','WAB'].mean()

In [ ]:
plt.figure(figsize=(12,10))

plt.subplot(321)
plt.scatter(y=datos['BARTHAG'], x=datos['W'],c=datos['cluster_j'],alpha=0.5, edgecolor='k')
plt.yticks(fontsize=12)
plt.ylabel('Prob Ganar', fontsize=12)
plt.title('Partidos ganados', fontsize=16)

plt.subplot(322)
plt.scatter(y=datos['BARTHAG'], x=datos['ADJOE'],c=datos['cluster_j'],alpha=0.5, edgecolor='k')
plt.yticks(fontsize=12)
plt.ylabel('Prob Ganar', fontsize=12)
plt.title('Eficiencia defensiva', fontsize=16)


plt.subplot(323)
plt.scatter(y=datos['BARTHAG'], x=datos['EFG_O'],c=datos['cluster_j'],alpha=0.5, edgecolor='k')
plt.yticks(fontsize=12)
plt.ylabel('Prob Ganar', fontsize=12)
plt.title('Lanzamientos efectivos', fontsize=16)


# Partidos ganados
plt.subplot(324)
plt.scatter(y=datos['BARTHAG'], x=datos['2P_O'],c=datos['cluster_j'],alpha=0.5, edgecolor='k')
plt.ylabel('Prob Ganar', fontsize=12)
plt.yticks(fontsize=12)
plt.title('Porcentaje de tiros de 2 puntos hechos', fontsize=16)


plt.subplot(325)
plt.scatter(y=datos['BARTHAG'], x=datos['WAB'],c=datos['cluster_j'],alpha=0.5, edgecolor='k')
plt.ylabel('Prob ganar', fontsize=12)
plt.yticks(fontsize=12)
plt.title('Triunfos por encima de la burbuja', fontsize=16)
plt.show()

## DBSCAN

Ajustamos también un DBSCAN a nuestros datos

In [ ]:
m = DBSCAN(eps=0.70, min_samples=10)
m.fit(km_scale)

In [ ]:
datos['cluster_db'] = m.labels_

In [ ]:
datos.cluster_db.value_counts()

In [ ]:
plt.scatter(datos['BARTHAG'], datos['W'], c=datos['cluster_db'])

En este caso particular DBSCAN no parece ser una buena solución. Sólo identifica 4 clusters pero la gran mayoría de puntos quedan identificados como noise points

## Ejercicios

1. Analizando los clusters formados por el k-means ¿Qué nombres le podríamos asignar a cada uno de los clusters que identifiquen sus comportamientos?
2. ¿Podríamos utilizar estos nombres para los resultados del jerárquico?
3. En el dendograma del agrupamiento jerárquico vimos que 5 clusters podría ser una solución. Ajustemos tanto para k-means como para el jerárquico nuevos agrupamientos utilizando 5 clusters. ¿Qué podemos concluir? ¿Cuál sería la mejor solución?

## Conclusiones

- Efectivamente se logran diferenciar tres grandes segmentos en los equipos caracterizados por su desempeño
- Se pueden identificar patrones en las métricas que permitan definir umbrales mínimos para los indicadores de rendimiento
- Se requiere de mayor información para escoger el número de clusters (más adelante lo veremos)